In [2]:
#encoding:utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

from string import punctuation
from collections import Counter
import re
df=pd.read_excel("SafeCity - Dashboard - Overall - Updated - April 2018_edited.xlsx")


In [4]:
for i,name in enumerate(df.columns):
    print i,name
print len(df)

0 INCIDENT ID
1 INCIDENT TITLE
2 DATE
3 YEAR
4 MONTH
5 WEEK OF THE MONTH
6 DAY
7 TIME
8 QUARTER OF THE DAY
9 HOUR OF THE DAY
10 MINUTE OF THE HOUR
11 DESCRIPTION
12 Catcalls/Whistles
13 Commenting
14 Sexual Invites
15 Ogling/Facial Expressions/Staring
16 Taking Pictures
17 Indecent Exposure
18 Touching /Groping
19 Stalking
20 Rape / Sexual Assault
21 Poor / No Street Lighting
22 Chain Snatching
23 North East India Report
24 Others
25 VERBAL ABUSE
26 NON-VERBAL ABUSE
27 PHYSICAL ABUSE
28 SERIOUS PHYSICAL ABUSE
29 OTHER ABUSE
30 Location
31 Country Code
32 City Code
33 Area Code
34 Locality Code
35 LATITUDE
36 LONGITUDE
37 Please Check
38 More Info
39 Flag
40 * Car *
41 * Train *
42 * Bus *
43 * Metro *
44 * Auto *
45 Taxi
46 Car
47 Train
48 Bus
49 Metro
50 Auto
51 * Rail *
52 Rail
53 Airport
54 Road
55 Travel
56 Passenger
57 Rickshaw
58 Driver
59 Conductor
60 Flag Car
61 Flag Train
62 Flag Bus
63 Flag Metro
64 Flag Auto
65 Flag Rail
66 Overall


10356

In [ ]:
df=df[["INCIDENT ID","INCIDENT TITLE","DESCRIPTION","LATITUDE","LONGITUDE","Location","City Code"]]

#only DESCRIPTION column contains NAs hence can omit all rows that contain NAs in the dataframe
df=df.dropna(axis=0,how='any') 

df.DESCRIPTION=map(lambda x: x.lower(),df.DESCRIPTION)

for text in df.DESCRIPTION:
    text=text.encode('utf-8')
    if type(text)=='unicode':
        a=a+1
        text=text.encode('utf-8')
        text=re.sub('[^A-Za-z0-9]+', ' ', text)
        
    print text



The reports were studied and keywords were identified to extract these recommendations. 
#### "Poorly lit area" was a very common pointer. 


In [ ]:
recommendations=[]
lat=[]
lon=[]

keywords=["no light","poor light","poorly lit","not safe","take care","not lit","unsafe","dangerous","bad light","patrol", 
         "avoid","whenever"]

for sent in df.DESCRIPTION:
    sentences=sent.split('.')
    s=""
    for sentence in sentences:
        
        sentence=sentence.strip('"')
        if any(x in sentence for x in keywords):
            #print sentence
            #print "###"
            if len(s)>0: s=s+","
            s=s+(sentence.encode('utf-8'))
            #removing \n etc
            s=s.replace('\n','')

    if len(s)>0:
        recommendations.append(s) 
        lat.append(df.iloc[row,2])
        lon.append(df.iloc[row,3])
    
#print (recommendations)
print len(recommendations),len(lat),len(lon)
recom=pd.DataFrame()

recom["RECOMMENDATION"]=recommendations
recom["LATITUDE"]=lat
recom["LONGITUDE"]=lon

#recom.to_csv("Recommendations_SafecityApril2018.csv",index=None)



It was found that some extracted data were not recommendations but incident reports, especially those associated with keyword "whenever". These were retained in the csv and weeded out later.


The<b> Address, City Code and Incident Title </b> were required as additional fields and hence had to be later appended to the recommendations sheet.

In [ ]:
# EXTRACTION OF ADDRESS, CITY CODE, INCIDENT TITLE

recom = pd.read_csv("Recommendations_SafecityApril2018.csv")


typeof=[]
address=[]
code_city=[]
print len(df_edited)
#print data["INCIDENT TITLE"]
for rec in df.RECOMMENDATION:
    
    if  len(df[df['DESCRIPTION'].str.contains(rec)])>0:
                
        x= (df[df['DESCRIPTION'].str.contains(rec)]).index.values[0]
        typeof.append(df.iloc[x]["INCIDENT TITLE"])
        address.append(df.iloc[x]["Location"])
        code_city.append(df.iloc[x]["City Code"])
    else:
        
        typeof.append("Not Available")
        address.append("Not Available")
        code_city.append("Not Available")

        
recom["INCIDENT TITLE"]=typeof
recom["ADDRESS"]=address
recom["CITY CODE"]=code_city


## Arranging the recommendations by City Codes (0 - Bangalore; 1 - Delhi; 2 - Mumbai)
df_new=pd.DataFrame(columns=list(recom))

df_new=(df[df["CITY CODE"]==0])
df_new=pd.concat([df_new,df[df["CITY CODE"]==1]],axis=0)
df_new=pd.concat([df_new,df[df["CITY CODE"]==2]],axis=0)
df_new=df_new.drop_duplicates(subset=['RECOMMENDATION'])

df_new.to_csv("Recommendations_Safecity2018_byCityCode.csv",index=None)


In [20]:
print list(df_new), df_new.shape
print df_new.head(6)

['Reports', 'LATITUDE', 'LONGITUDE', 'INCIDENT TITLE', 'ADDRESS', 'CITY CODE'] (112, 6)
                                             Reports   LATITUDE  LONGITUDE  \
0  in sarjapura road signal bus stop bangalore th...  12.971599  77.594563   
1  unsafe sub way poorly lit light without any se...  13.026604  77.585916   
2  it is properly lighted it is extremely busy it...  13.060422  80.249583   
3   the area around nageshwara park is also not s...  13.033611  80.268611   
4  according to me the most unsafe place in delhi...  20.429047  80.149403   
5  at night iwas passing through moti bagh i foun...  20.429047  80.149403   

                  INCIDENT TITLE  \
0               Beware of pimps.   
1               Touching/Groping   
2           Harrowing experience   
3               Isolated subways   
4  Commenting/ Cat Calls/ Ogling   
5                     COMMENTING   

                                             ADDRESS  CITY CODE  
0  Grant Rd, KG Halli, Shanthala Nagar, Sampan

<b> The extracted recommendations were submitted to Safecity and is to be compiled into a white paper. </b>